Copyright (c) 2025–2026 Quan-feng WU <wuquanfeng@ihep.ac.cn>

This notebook is released under the [MIT License](https://opensource.org/licenses/MIT).

# Preliminaries

In [ ]:
using CairoMakie
using DelimitedFiles
using Format
using JLD2
using LaTeXStrings
using SpecialFunctions

using NaturalUnits

In [ ]:
include("tool_script-directories.jl")
include("tool_script-parameters.jl")
include("tool_script-geomspace.jl")

@info "Loaded tool scirpts."

In [ ]:
set_theme!(theme_latexfonts())

In [ ]:
fmt_1f = "%.1f"
fmt_2e = "%.2e"
m4_ticks = [0.1, 0.2, 0.5, 1, 2, 5, 10]
m4_tick_format = vs->["$(cfmt(fmt_1f, v))" for v in vs]

NU = NaturalUnit(GeV)

## Data Loading

In [ ]:
results_data = (load ∘ joinpath)(output_data_directory, "results.jld2")

m₄_list = results_data["m4 [GeV]"]
U_e4_norm_sqr_list = results_data["U_e4_norm"].^2

m₄_min = minimum(m₄_list)
m₄_max = maximum(m₄_list)
U_e4_norm_sqr_min = minimum(U_e4_norm_sqr_list)
U_e4_norm_sqr_max = maximum(U_e4_norm_sqr_list)

@info "Results loaded with keys:\n    \"$(join((sort! ∘ collect ∘ keys)(results_data), "\",\n    \""))\"."

# Evolution Plots

In [ ]:
ii, jj, Brₓ = 45, 7, 0.5
m₄ = m₄_list[ii]
U_e4_norm_sqr = U_e4_norm_sqr_list[jj]

SBBN_data = (load ∘ joinpath)(output_data_directory, "dump_data", "dump_data_SBBN.jld2")
@info "Loaded SBBN data with keys:\n    \"$(join((sort! ∘ collect ∘ keys)(SBBN_data), "\",\n    \""))\"."

dump_data = (load ∘ joinpath)(output_data_directory, "dump_data",
    "dump_data_$(cfmt(fmt_2e, m₄))GeV_$(cfmt(fmt_2e, U_e4_norm_sqr))_$(cfmt(fmt_1f, Brₓ * 100))percent.jld2"
)
@info "Loaded dump data for m₄ = $(m₄) GeV, |Uₑ₄|² = $(U_e4_norm_sqr), Brₓ = $(Brₓ * 100)% with keys:\n    \"$(join((sort! ∘ collect ∘ keys)(dump_data), "\",\n    \""))\"."

In [ ]:
fig = Figure(size=(1000, 400))
ax_1 = Axis(fig[1, 1],
    title=L"$X_n$ Evolution",
    xlabel=L"$T$ [MeV]", ylabel=L"$X_n$",
    xscale=log10, # yscale=log10,
    xreversed=true,
    limits=((1e-2, 1e2), (-1e-2, .6)),
    yticks=-1:.1:1,
)
lines!(ax_1, 1e3 * SBBN_data["T_SM [GeV]"], SBBN_data["X_n"]; label="SBBN")
lines!(ax_1, 1e3 * dump_data["T_SM [GeV]"], dump_data["X_n"]; label="With HNL")
lines!(ax_1, EUval.(MeV, [T_nuc, T_nuc]), [-0, 1]; linestyle=:dash)
axislegend(ax_1; position=:lb)

ax_2 = Axis(fig[1, 2],
    title="Energy Density Evolution",
    xlabel=L"$T$ [MeV]", ylabel=L"$\rho / \rho_\nu$",
    xscale=log10, yscale=log10,
    xticks=exp10.(-10:10), xtickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
    yticks=exp10.(-10:10), ytickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
    xreversed=true,
    limits=((1e-2, maximum(1e3 .* dump_data["T_SM [GeV]"])), (1e-4, 1e2))
)
lines!(ax_2, 1e3 .* dump_data["T_SM [GeV]"],
    (dump_data["rho_SM [GeV^4]"] .+ dump_data["rho_4 [GeV^4]"] .+ dump_data["rho_X [GeV^4]"]) ./ dump_data["rho_nu [GeV^4]"];
    label=L"\rho_\mathrm{tot} / \rho_\nu",
)
lines!(ax_2, 1e3 .* dump_data["T_SM [GeV]"], dump_data["rho_SM [GeV^4]"] ./ dump_data["rho_nu [GeV^4]"]; label=L"\rho_\mathrm{SM} / \rho_\nu",)
lines!(ax_2, 1e3 .* dump_data["T_SM [GeV]"],
    (dump_data["rho_4 [GeV^4]"] .+ dump_data["rho_X [GeV^4]"]) ./ dump_data["rho_nu [GeV^4]"];
    label=L"\left(\rho_N + \rho_X\right) / \rho_\nu", linestyle=:dash,
)
lines!(ax_2, 1e3 .* dump_data["T_SM [GeV]"], dump_data["rho_4 [GeV^4]"] ./ dump_data["rho_nu [GeV^4]"]; label=L"\rho_N / \rho_\nu", linestyle=:dash)
lines!(ax_2, 1e3 .* dump_data["T_SM [GeV]"], dump_data["rho_X [GeV^4]"] ./ dump_data["rho_nu [GeV^4]"]; label=L"\rho_X / \rho_\nu", linestyle=:dash)
lines!(ax_2, EUval.(MeV, [T_nuc, T_nuc]), [1e-5, 1e6]; linestyle=:dash)
axislegend(ax_2; position=:rb)
fig

# Output Plots

In [ ]:
# fig_out_1 = Figure(size=(600, 450))
fig_out_1 = Figure(size=(1500, 1800), fontsize=24)

Brₓ_str_list = ["10.0%", "50.0%", "90.0%", "99.0%"]
for ii ∈ eachindex(Brₓ_str_list)
    Brₓ_str = Brₓ_str_list[ii]

    ax_ΔN_eff = Axis(fig_out_1[ii, 1][1, 1],
        xlabel=ii == length(Brₓ_str_list) ? L"m_N~[\mathrm{GeV}]" : "",
        ylabel=L"U_e^2",
        xlabelsize=32, ylabelsize=32,
        xscale=log10, yscale=log10,
        xticks=m4_ticks, xtickformat=m4_tick_format,
        yticks=exp10.(-13:-6), ytickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
        limits=((m₄_min, 1e1), (1e-13, 1e-6))
    )
    con_ΔN_eff = contourf!(ax_ΔN_eff, m₄_list, U_e4_norm_sqr_list, results_data["Delta_N_eff for Br_x = $Brₓ_str"];
        levels=0:.05:.6,
        colormap=:viridis,
        extendhigh=:yellow,
    )
    Colorbar(fig_out_1[ii, 1][1, 2], con_ΔN_eff; label=L"\Delta N_\mathrm{eff}^\mathrm{CMB}", ticks=0:.1:.6)
    text!(ax_ΔN_eff, .6, 1e-12; text=L"\mathrm{Br}_X = %$Brₓ_str")

    ax_Yₚ = Axis(fig_out_1[ii, 2][1, 1],
        xlabel=ii == length(Brₓ_str_list) ? L"m_N~[\mathrm{GeV}]" : "",
        ylabel=L"U_e^2",
        xlabelsize=32, ylabelsize=32,
        xscale=log10, yscale=log10,
        xticks=m4_ticks, xtickformat=m4_tick_format,
        yticks=exp10.(-13:-6), ytickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
        limits=((m₄_min, 1e1), (1e-13, 1e-6))
    )
    con_Yₚ = contourf!(ax_Yₚ, m₄_list, U_e4_norm_sqr_list, results_data["Y_P for Br_x = $Brₓ_str"] .- .245;
        levels=0:.002:.024,
        colormap=:viridis,
        extendhigh=:yellow,
        extendlow=RGBf(.2, 0, .2)
    )
    Colorbar(fig_out_1[ii, 2][1, 2], con_Yₚ; label=L"\Delta Y_P", ticks=0:.004:.024)
    text!(ax_Yₚ, .6, 1e-12; text=L"\mathrm{Br}_X = %$Brₓ_str")
end

save(joinpath(plot_directory, "Neff-and-Yp.pdf"), fig_out_1)
fig_out_1

In [ ]:
# fig_out_2 = Figure(size=(600, 450))
fig_out_2 = Figure(size=(600, 900), fontsize=16)

typeI_lower_list, typeI_super_list = .01 * 1e-9 ./ m₄_list, .1 * 1e-9 ./ m₄_list
current_LNC_data = exp10.(readdlm(joinpath(external_data_directory, "current_LNC_data.csv"), ','))
current_LNV_data = exp10.(readdlm(joinpath(external_data_directory, "current_LNV_data.csv"), ','))
DUNE_1_data = exp10.(readdlm(joinpath(external_data_directory, "DUNE_1_data.csv"), ','))
DUNE_2_data = exp10.(readdlm(joinpath(external_data_directory, "DUNE_2_data.csv"), ','))
PIONEER_data = exp10.(readdlm(joinpath(external_data_directory, "PIONEER_data.csv"), ','))
SHiP_data = exp10.(readdlm(joinpath(external_data_directory, "SHiP_data.csv"), ','))
current_0νββ_data = readdlm(joinpath(external_data_directory, "current_0νββ_data.csv"), ',')
future_0νββ_data = readdlm(joinpath(external_data_directory, "future_0νββ_data.csv"), ',')

Brₓ_str_list = ["10.0%", "50.0%", "90.0%", "99.0%"]
χ²_data_Brₓ0 = (results_data["Delta_N_eff for Br_x = 0.0%"] ./ .15).^2 + ((results_data["Y_P for Br_x = 0.0%"] .- .245) ./ .003).^2
χ²_label_position = [(2, 6e-11), (5., 1.6e-10), (5., 2e-10), (5., 7e-11)]
neutrinoless_double_beta_label_position = [(3, 3e-8), (2.2, 2.2e-8), (3.2, 3.2e-8), (2.5, 2.5e-8)]
ax_1 = nothing
con_1 = nothing
for ii ∈ eachindex(Brₓ_str_list)
    Brₓ_str = Brₓ_str_list[ii]
    χ²_data = (results_data["Delta_N_eff for Br_x = $Brₓ_str"] ./ .15).^2 + ((results_data["Y_P for Br_x = $Brₓ_str"] .- .245) ./ .003).^2

    # plt = fig_out_2[ii + 1, 1]
    ax = Axis(fig_out_2[ii+1, 1],
        xlabel=ii == length(Brₓ_str_list) ? L"m_N~[\mathrm{GeV}]" : "",
        ylabel=L"U_e^2",
        xlabelsize=20, ylabelsize=20,
        xscale=log10, yscale=log10,
        xticks=m4_ticks, xtickformat=m4_tick_format,
        xticklabelsvisible=ii == length(Brₓ_str_list),
        yticks=exp10.(-13:-6), ytickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
        limits=((m₄_min, 1e1), (1e-13, 1e-6))
    )
    con = contourf!(ax, m₄_list, U_e4_norm_sqr_list, χ²_data;
        levels=0:24,
        colormap=:viridis,
        extendhigh=:yellow
    )

    if ii == 1
        ax_1 = ax
        con_1 = con
    end

    contour!(ax, m₄_list, U_e4_norm_sqr_list, χ²_data;
        levels=[4],
        color=:white,
        linestyle=:dash,
        linewidth=2
    )
    text!(ax, χ²_label_position[ii]...; text=L"2 \sigma", color=:white, rotation=-.34)

    contour!(ax, m₄_list, U_e4_norm_sqr_list, χ²_data_Brₓ0;
        levels=[4],
        color=:blue,
        linestyle=:dash,
        linewidth=2
    )
    text!(ax, 1.7, 4.5e-12; text=L"$2 \sigma$ for $\mathrm{Br}_X = 0$", color=:blue, rotation=-.48)

    band!(ax, m₄_list, typeI_lower_list, typeI_super_list; color=(:green, 0.3))
    text!(ax, .6, 3e-11; text="Type-I Seesaw", rotation=-.11, color=:green)

    lines!(ax, current_LNC_data[:, 1], current_LNC_data[:, 2]; color=:black)
    band!(ax, current_LNC_data[:, 1], current_LNC_data[:, 2], [1e-6 for _ ∈ current_LNC_data[:, 1]]; color=(:gray, 0.5))
    # lines!(ax, current_LNV_data[:, 1], current_LNV_data[:, 2]; color=:black)
    # band!(ax, current_LNV_data[:, 1], current_LNV_data[:, 2], [1e-6 for _ ∈ current_LNV_data[:, 1]]; color=(:gray, 0.5))
    text!(ax, .15, 4e-8; text="Current", color=:black)

    lines!(ax, current_0νββ_data[:, 1], current_0νββ_data[:, 2]; color=:deeppink3)
    text!(ax, 0.13, 2e-10; text=L"0\nu\beta\beta", color=:deeppink3, rotation=.09)
    # lines!(ax, future_0νββ_data[:, 1], future_0νββ_data[:, 2]; color=:lightslateblue)
    # text!(ax, 0.17, 5e-11; text=L"0\nu\beta\beta", color=:lightslateblue, rotation=.09)

    lines!(ax, DUNE_2_data[:, 1], DUNE_2_data[:, 2]; label="DUNE", color=:maroon1,)
    text!(ax, .3, 2e-11; text="DUNE", color=:maroon1)

    lines!(ax, PIONEER_data[:, 1], PIONEER_data[:, 2]; label="PIONEER", color=:lightslateblue,)
    text!(ax, .12, 3e-11; text="PIONEER", color=:lightslateblue)

    lines!(ax, SHiP_data[:, 1], SHiP_data[:, 2]; label="SHiP", color=:red,)
    # text!(ax, .6, 2.5e-10; text="SHiP", color=:red, rotation=-.2)
    text!(ax, .75, 1.75e-10; text="SHiP", color=:red, rotation=-.3)

    text!(ax, .7, 2e-13; text=L"\mathrm{Br}_X = %$Brₓ_str")

    lines!(ax, m₄_list, 3e-12 .* sqrt(106.75 / 83.5) ./ m₄_list; label="Thermalization", color=:orangered, linestyle=:dot)
    text!(ax, 0.2, 3e-12; text="Thermalization", color=:orangered, rotation=-.098)
    # lines!(ax, m₄_list, 4 ./ (G_F .* M_Pl .* GeV.(m₄_list)); label="Thermalization", color=:orangered, linestyle=:dot)
    # text!(ax, 0.15, 1e-12; text="Thermalization", color=:orangered, rotation=-.099)
end

Colorbar(fig_out_2[1, 1], con_1; vertical=false)
text!(fig_out_2.scene, .1, .94; text=L"\chi^2", fontsize=20, space=:relative)

save(joinpath(plot_directory, "chi2.pdf"), fig_out_2)
fig_out_2

In [ ]:
fig_out_3 = Figure(fontsize=18)

τ_in_s = (inv ∘ GeV).(results_data["Gamma_SM [GeV] for Br_x = 0.0%"]) ./ NU.s

ax_3 = Axis(fig_out_3[1, 1],
    xlabel=L"m_N~[\mathrm{GeV}]",
    ylabel=L"U_e^2",
    xscale=log10, yscale=log10,
    xticks=m4_ticks, xtickformat=m4_tick_format,
    yticks=exp10.(-13:-6), ytickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
    limits=((m₄_min, 1e1), (1e-13, 1e-6))
)

con_3 = contourf!(ax_3, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=0.:.01:.1,
    colormap=:viridis,
    extendhigh=:yellow,
    # extendlow=RGBf(.2, 0, .2)
)
Colorbar(fig_out_3[1, 2], con_3; label=L"$\tau_N$ [sec]", ticks=0:.01:.1)

contour!(ax_3, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=[0.02],
    color=[:white],
    linestyle=:dash, linewidth=2
)
text!(ax_3, 1., 3e-10; text=L"\tau_N = 0.02 ~ \mathrm{sec}", color=:white, rotation=-.98)

contour!(ax_3, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=[0.1],
    color=[:black],
    linestyle=:dashdot, linewidth=2
)
text!(ax_3, .68, 5e-11; text=L"\tau_N = 0.1 ~ \mathrm{sec}", color=:black, rotation=-.95)

save(joinpath(plot_directory, "lifetime.pdf"), fig_out_3)
fig_out_3

In [ ]:
fig_out_4 = Figure(size=(1200, 450), fontsize=18)

Ue_sqrd_boundary_1 = readdlm(joinpath(external_data_directory, "Ue_sqrd_boundary-from-2006.07387.csv"), ',')
Ue_sqrd_boundary_2 = readdlm(joinpath(external_data_directory, "Ue_sqrd_boundary-from-2008.00749.csv"), ',')
Ue_sqrd_MD_boundary = readdlm(joinpath(external_data_directory, "Ue_sqrd_MD_boundary.csv"), ',')

τ_in_s = (inv ∘ GeV).(results_data["Gamma_SM [GeV] for Br_x = 0.0%"]) ./ NU.s
log10_τ_in_s = log10.(τ_in_s)

plt_left = fig_out_4[1, 1]
ax_left = Axis(plt_left[2, 1],
    xlabel=L"m_N~[\mathrm{GeV}]",
    ylabel=L"U_e^2",
    xlabelsize=20, ylabelsize=20,
    xscale=log10, yscale=log10,
    xticks=m4_ticks, xtickformat=m4_tick_format,
    yticks=exp10.(-13:-6), ytickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
    limits=((m₄_min, 1e1), (1e-13, 1e-6))
)

con = contourf!(ax_left, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=0.:.01:.1,
    colormap=:viridis,
    extendhigh=:yellow,
    extendlow=RGBf(.2, 0, .2)
)
Colorbar(plt_left[1, 1], con; ticks=0:.01:.1, vertical=false)

lines!(ax_left, Ue_sqrd_boundary_1[:, 1], Ue_sqrd_boundary_1[:, 2]; linewidth=3, label="From Sabti et al. (2020)", color=:red)
lines!(ax_left, Ue_sqrd_boundary_2[:, 1], Ue_sqrd_boundary_2[:, 2]; linewidth=3, label="From Boyarsky et al. (2021)", color=:magenta)
lines!(ax_left, Ue_sqrd_MD_boundary[:, 1], Ue_sqrd_MD_boundary[:, 2]; linewidth=3, label="Temporary MD Boundary", color=:purple, linestyle=:dashdot)

contour!(ax_left, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=[0.02],
    color=[:white],
    linestyle=:dash, linewidth=3
)
text!(ax_left, 1., 3e-10; text=L"\tau_N = 0.02 ~ \mathrm{sec}", color=:white, rotation=-.76)

contour!(ax_left, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=[0.1],
    color=[:black],
    linestyle=:dash, linewidth=3
)
text!(ax_left, .4, 5e-10; text=L"\tau_N = 0.1 ~ \mathrm{sec}", color=:black, rotation=-.6)
axislegend(ax_left; position=:lb, backgroundcolor=:transparent)

plt_right = fig_out_4[1, 2]
ax_right = Axis(plt_right[2, 1],
    xlabel=L"m_N~[\mathrm{GeV}]",
    ylabel=L"U_e^2", ylabelcolor=(:black, .0),
    xlabelsize=20, ylabelsize=10,
    xscale=log10, yscale=log10,
    xticks=m4_ticks, xtickformat=m4_tick_format,
    yticks=exp10.(-13:-6), ytickformat=vs->["" for v ∈ vs], #vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
    limits=((m₄_min, 1e1), (1e-13, 1e-6))
)
con = contourf!(ax_right, m₄_list, U_e4_norm_sqr_list, log10_τ_in_s;
    levels=-10:7,
    colormap=:viridis,
    extendhigh=:yellow,
    extendlow=RGBf(.2, 0, .2)
)
Colorbar(plt_right[1, 1], con; ticks=(-10:2:7, [L"10^{%$(v)}" for v in -10:2:7]), vertical=false)

lines!(ax_right, Ue_sqrd_boundary_1[:, 1], Ue_sqrd_boundary_1[:, 2]; linewidth=3, label=L"", color=:red)
lines!(ax_right, Ue_sqrd_boundary_2[:, 1], Ue_sqrd_boundary_2[:, 2]; linewidth=3, label=L"", color=:magenta)
lines!(ax_right, Ue_sqrd_MD_boundary[:, 1], Ue_sqrd_MD_boundary[:, 2]; linewidth=3, color=:purple, linestyle=:dashdot)

contour!(ax_right, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=[0.02],
    color=[:white],
    linestyle=:dash, linewidth=3
)
text!(ax_right, 1., 3e-10; text=L"\tau_N = 0.02 ~ \mathrm{sec}", color=:white, rotation=-.76)

contour!(ax_right, m₄_list, U_e4_norm_sqr_list, τ_in_s;
    levels=[0.1],
    color=[:black],
    linestyle=:dash, linewidth=3
)
text!(ax_right, .4, 5e-10; text=L"\tau_N = 0.1 ~ \mathrm{sec}", color=:black, rotation=-.6)

text!(fig_out_4.scene, .02, .88; text=L"$\tau_N$ [sec]", space=:relative, fontsize=20)

save(joinpath(plot_directory, "lifetime-new.pdf"), fig_out_4)
fig_out_4